In [1]:
import os
import tensorflow as tf

In [2]:
file_name = "index_HanDB_Train.tfrecord"
record_iterator = tf.python_io.tf_record_iterator(path=file_name)

numb_of_data=0
for record in record_iterator:
    numb_of_data+=1

tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./"+file_name))
#f=open("./test_record.tfrecord",'rb')

In [3]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int64)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels

In [4]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_labels = tf.placeholder(tf.int64,[None])
keep_prob = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)

Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)
Tensor("Placeholder_1:0", shape=(?,), dtype=int64)


In [5]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=True)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [7]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=True)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=True)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [9]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=True)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)


In [10]:
W5 = tf.Variable(tf.random_normal([4*4*256, 1024], stddev=0.01))
fc1 = tf.reshape(pool4,[-1,4*4*256])
fc1 = tf.matmul(fc1, W5)
fc1 = tf.nn.relu(fc1)

fc1 = tf.nn.dropout(fc1,keep_prob)
print(fc1)

Tensor("dropout/mul:0", shape=(?, 1024), dtype=float32)


In [11]:
W6 = tf.Variable(tf.random_normal([1024,2350],stddev=0.01))
logit = tf.matmul(fc1, W6)
print(logit)

Tensor("MatMul_1:0", shape=(?, 2350), dtype=float32)


In [12]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)
print(loss)

Tensor("Mean:0", shape=(), dtype=float32)


In [13]:
label_max=tf.argmax(logit,1)
print(label_max)

correct_prediction = tf.equal(tf.argmax(logit,1),input_labels)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [20]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()
batch_size=100

total_batch = int(numb_of_data / batch_size)
images, labels = read_and_decode(tf_record_filename_queue,batch_size)
print("Total Batch :%d" % (total_batch))
with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for epoch in range(100):
        for i in range(total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels = sess.run([images,labels])
            sess.run([optimizer,loss],feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:0.5})
            train_acc=sess.run(accuracy, feed_dict={input_images: batch_images, input_labels:batch_labels, keep_prob:1.0})
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy %f' % (epoch,i, train_acc))
        save_path = saver.save(sess, "save\model2.ckpt")
    coord.request_stop()
    coord.join(threads)

Total Batch :6658
Epoch :0 | step 0, training accuracy 0.030000
Epoch :0 | step 100, training accuracy 0.020000
Epoch :0 | step 200, training accuracy 0.000000
Epoch :0 | step 300, training accuracy 0.090000
Epoch :0 | step 400, training accuracy 0.160000
Epoch :0 | step 500, training accuracy 0.200000
Epoch :0 | step 600, training accuracy 0.190000
Epoch :0 | step 700, training accuracy 0.420000
Epoch :0 | step 800, training accuracy 0.400000
INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.InvalidArgumentError'>, Input to reshape is a tensor with 1 values, but the requested shape has 4096
	 [[Node: Reshape_6 = Reshape[T=DT_FLOAT, Tshape=DT_INT32, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Cast_11, Reshape_6/shape)]]
	 [[Node: Reshape_6/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_na

OutOfRangeError: RandomShuffleQueue '_41_shuffle_batch_5/random_shuffle_queue' is closed and has insufficient elements (requested 100, current size 11)
	 [[Node: shuffle_batch_5 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT64], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch_5/random_shuffle_queue, shuffle_batch_5/n)]]

Caused by op 'shuffle_batch_5', defined at:
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykern

In [8]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

batch_size=100

total_batch = int(numb_of_data / batch_size)
images, labels = read_and_decode(tf_record_filename_queue,batch_size)

with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    for i in range(total_batch):
        img, lab = sess.run([images,labels])
        if(i>800):
            print("Batch %d" % (i))
            print(lab)
    coord.request_stop()
    coord.join(threads)


Batch 801
[ 294  299 1255 1395  633 1550  741 1640 1214 1825  223 1548 1682 1486
   86 1807  560 1889 1625 1597 1834 1505 2225 1597  152  839 1335  313
  386  364  163  828 1030  822  253 1851  540 1851 1118 1689  908  986
 2137   91  803 1527 2008  148 1839  347 1487 2259  691  365  964  933
  722  765 2221 1020  681  903 1217 1112 2210  907  478 1829  647 1598
 1593  325 1413 1822  774 2291  602 1370 1267 1410 1518  407   42 1084
   64  933 1138 1398 1430 2269 2269  416  954 1780 2081  246  835 1177
 1418 1001]
Batch 802
[2154  839 2271 1652  770  755  939  374 1279  568  672 1596 1691 1380
 1410 2132    7 1779 1031 1897 1778  416  112  494   44 1689 1022 1498
  650  518  298  697   13 1366 1682 1109 1116  766 2216 1920  954  986
 2292 1896   18  542  508 1393  242  536 1685 1497  244  126 1652 1682
 2018 2253  356 1213 1848  361 2247 2203 1257  509 1126 2258 1418  165
 1363 1530 1787 1910 2294 1249 1565 2148  347 1317 1519 2279 2209 1547
 1603 1787 2327 1933  788  690  349  813 1238

Batch 819
[1690 1002 2251 2266  361 1704  490   62  304 1263 2279  264  112  751
  649 1627  822 2116 2218  774 1533  427  651 1104   36  788 2260 1550
 2007 2270 1232 2331 1788 2211  358 1418   76  409  832  754 2346 1180
 1217 1266  666 2082 1723 1491 2230  918  155  781   51  452  138  359
  913 1370 2017 1691 1497  758 1596  310  989 2095  199 1266 2171  386
 2053 1498  639 1404 1555 1487  102  302 1356 1469  900 1478 1486 1685
  205  606  699 1112  906 2217 1582 1455  121 1601  831 1807 2225 1780
  435 1146]
Batch 820
[ 128 1997  743  989 1547   87  329  651 2208   71  409  775    0  461
  118 1335  510  717 2236 1358 2046 1478 1603 1264  450  651 1880  277
 2349  650 2144  155  132  465 1582 2058  521 1714    4  954  103  368
 1370  746 1527 1397  729  539  935  443  291 1248 1141 1150 1094 1456
  145  515 1031  980 1630  651 1780  118  563  903 1084 1392  568 1022
  938 1689 1603 1812 1532 2208 1480 2253  492  540 1020 1594  590 1104
  191 1620 1084 1805  856 2261  443 1478 1488

Batch 837
[ 938 1707  461   97 1812 2210  623 1212 1582 2166  542  358  560  768
  964 1533  364 1485  715  906 1783   40   95 1488  937  144   87 1566
  150 1141  490  705   18   36 1353  424 1541 2266  154  440    1 1368
   13  893   80  633 1259 1252 2217 1355  171  964 1154 1368  300  386
  776 1880  592 1444 1423 1563 1984 1487 1333 1425 2336 2266  275 1548
 1477   80  329    7 1986  654  244  435   18  104 2007  547 1561  985
  959 1502  508  508  540 1997  903  703 1250 2083  774  697  356 1491
  967  299]
Batch 838
[ 579 1398  804 2210 1827 2007 1498 1640 1259  774   92 1562 2271  647
 2025 2269   91 2083 1535 1706  539 1525  758 1404  698  359 1685  775
   36  893  915  364  728 2245  867  954 1646 1368  868 1397    4   52
 1452  349  648 1827  391   97 1787   64  120 1244 1879  336 2107  831
 1267 1692  904 1394 1494  906 1418  375 1412 2309 1654 1890  760  767
  620  140 2217 2115 1682  112 2322  878  650 1498 1597 1333  864 1593
  998 1630  935 1253 1721 2008 1653 1600  474

OutOfRangeError: RandomShuffleQueue '_14_shuffle_batch_2/random_shuffle_queue' is closed and has insufficient elements (requested 100, current size 11)
	 [[Node: shuffle_batch_2 = QueueDequeueManyV2[component_types=[DT_FLOAT, DT_INT64], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch_2/random_shuffle_queue, shuffle_batch_2/n)]]

Caused by op 'shuffle_batch_2', defined at:
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykern